# DataJam (NYC Dept of City Planning, American Planning Association, BetaNYC) Challenge on May 8th 2017 

### Challenge:

The goal is to *__identify duplicate records__* in the [Facilities Database](https://www1.nyc.gov/site/planning/data-maps/open-data/dwn-selfac.page) of the NYC Department of Capital Planning. 

This dataset contains information about facilities and program sites that are owned, operated, funded, licensed or certified by a City, State, or Federal agency in New York City. The DCP recently built the [Facilties Explorer](https://capitalplanning.nyc.gov/facilities), an interactive map based visualization tool that allows ease of access of this dataset and helps one filter data based on requirements.

However, after preliminary investigation  and understanding of this tool, one will observe certain duplicate entries for the same location. Examples provided for this DataJam by the NYC_DCP are as follows:
1. [Polly Dodge Early Learning Center](https://capitalplanning.nyc.gov/facilities/explorer#20/40.76832/-73.99177)
2. [Harlem Children's Zone](https://capitalplanning.nyc.gov/facilities/explorer#20/40.81256/-73.94825)
3. [Bronx City Recycling](https://capitalplanning.nyc.gov/facilities/explorer#20/40.80724/-73.88005)

### Approach:

The approach followed to Identify and Eliminate duplicate records in this notebook is as follows:
* A. Observe the available data for the 3 examples
* B. Eliminate rows that are an exact duplicate of another row.
* C. Eliminate rows that do not have a name or address for a facility
* D. Evaluate cleaned data for the same 3 examples
* E. Test fuzzywuzzy library implementation on the cleaned data for the same 3 examples

### A. Observe the available data for the 3 examples

In [1]:
#Importing pandas
import pandas as pd

We now load the already downloaded facilites dataset

In [2]:
facilities = pd.read_csv('facilities.csv')
facilities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35574 entries, 0 to 35573
Data columns (total 47 columns):
uid            35574 non-null int64
hash           35574 non-null object
idold          189 non-null object
idagency       24599 non-null object
facname        35567 non-null object
addressnum     33130 non-null object
streetname     33130 non-null object
address        35040 non-null object
city           35542 non-null object
boro           35574 non-null object
borocode       35574 non-null int64
zipcode        35426 non-null float64
latitude       35574 non-null float64
longitude      35574 non-null float64
xcoord         35574 non-null float64
ycoord         35574 non-null float64
bin            31341 non-null object
bbl            34246 non-null object
commboard      35385 non-null float64
council        35574 non-null int64
censtract      35574 non-null int64
nta            35385 non-null object
facdomain      35574 non-null object
facgroup       35574 non-null object
fac

In [3]:
#View first 5 rows
facilities.head()

,uid,hash,idold,idagency,facname,addressnum,streetname,address,city,boro,...,overagency,overabbrev,datasource,dataname,datalink,datadate,pgtable,uid_merged,hash_merged,geom
0,33492,a789c225b1ed04091a7eb14a93c8253c,NaN,NYCDCA: 1430208-DCA,Ud Parking Corp.,761,East 168 Street,761 East 168 Street,Bronx,Bronx,...,NYC Department of Consumer Affairs,NYCDCA,NYCDCA,NYCDCA: Legally Operating Businesses,NYCDCA: https://data.cityofnewyork.us/Business...,NYCDCA: 2017-01-20,dca_facilities_operatingbusinesses,NaN,NaN,0101000020E6100000FD37F6A1A97952C008247B61306A...
1,11451,77ec8a8ffdbe776aeeed1c37d6de9f34,NaN,NYCDOE: K274,P.S. 274 Kosciusko,800,Bushwick Avenue,800 Bushwick Avenue,Brooklyn,Brooklyn,...,NYC Department of Education,NYCDOE,NaN,:,:,:,doe_facilities_universalprek;dcas_facilities_colp,NaN,NaN,0101000020E6100000934B4BCC637B52C0A8AB7463E258...
2,40506,64e3f413da65472ca5e39d7b31c02c95,NaN,NYSOMH: 872099-7796-001,Brooklyn Mental Health Court,320,Jay Street,320 Jay Street,Brooklyn,Brooklyn,...,NYS Office of Mental Health,NYSOMH,NYSOMH,NYSOMH: Local Mental Health Programs,NYSOMH: https://data.ny.gov/Human-Services/Loc...,NYSOMH: 2016-10-24,nysomh_facilities_mentalhealth,NaN,NaN,0101000020E61000000E172F93367F52C0DB735816E958...
3,16153,fef53b0de14124f1e69037d4a04add59,NaN,NYCDOE: K204,P.S. 204 - K,8101,15 Avenue,8101 15 Avenue,Brooklyn,Brooklyn,...,NYC Department of Education,NYCDOE,NaN,:,:,:,dcas_facilities_colp;doe_facilities_schoolsblu...,16154,699f2d2b70a5bb8e13663b04eeb796bb,0101000020E610000028EAC645688052C097A69F298D4E...
4,11665,04f0545585eaaef29a3e505d18b3014a,NaN,NaN,Beach Channel Drive,NaN,NaN,Beach 108 Street,Rockaway Park,Queens,...,NYC Department of Environmental Protection,NYCDEP,NYCDCAS,NYCDCAS: City Owned and Leased Properties,NYCDCAS: http://www1.nyc.gov/site/planning/dat...,NYCDCAS: 2016-10-20,dcas_facilities_colp,5357,160c18a9e48bf3879daddf618e03eba2,0101000020E6100000FA59D0D72E7552C07C88A81CC74A...


#### Data snippet for example 1

In [4]:
polly = facilities[facilities['bin']=='1086807']
polly

,uid,hash,idold,idagency,facname,addressnum,streetname,address,city,boro,...,overagency,overabbrev,datasource,dataname,datalink,datadate,pgtable,uid_merged,hash_merged,geom
5815,37687,27ae6e2641c1a5d1bd44e8ded81ce2d9,NaN,NYCDOE: MAUT;NYCACS: 704704,YWCA - NYC Polly Dodge ELC,538,West 55 Street,538 West 55 Street,New York,Manhattan,...,NYC Administration for Childrens Services;NYC ...,NYCACS;NYCDOE,NaN,:,:,:,acs_facilities_daycareheadstart;doe_facilities...,c7cbf9d4e16bd174521ac18ebcae7ff2,c7cbf9d4e16bd174521ac18ebcae7ff2,0101000020E61000004C981A9F777F52C07DF58B995862...
6233,49518,84603ad08140caef760a522bd8635005,NaN,NYCHHS: CT106820171403865,Polly Dodge Early Learning Center,538,West 55 Street,538 West 55 Street,New York,Manhattan,...,NYC Administration for Childrens Services,NYCACS,NaN,:,:,:,hhs_facilities_financialscontracts,NaN,NaN,0101000020E61000004C981A9F777F52C07DF58B995862...
7176,49517,29cae0fe8f633cca1abaa9b9047328ac,NaN,NYCHHS: CT106820171403865,Polly Dodge Early Learning Center,538,West 55 Street,538 West 55 Street,New York,Manhattan,...,NYC Administration for Childrens Services,NYCACS,NaN,:,:,:,hhs_facilities_financialscontracts,NaN,NaN,0101000020E61000004C981A9F777F52C07DF58B995862...
12156,37691,4d5620b62186d283b6e3b938400f6997,NaN,NYCDOHMH: DC3066,Young Women's Christian Association Of The Cit...,538,West 55 Street,538 West 55 Street,New York,Manhattan,...,NYC Department of Health and Mental Hygiene,NYCDOHMH,NYCDOHMH,NYCDOHMH: DOHMH Childcare Center Inspections,NYCDOHMH: https://data.cityofnewyork.us/Health...,NYCDOHMH: 2017-01-23,dohmh_facilities_daycare,NaN,NaN,0101000020E61000004C981A9F777F52C07DF58B995862...
27606,49515,9b3f4bfd1da264aebf00e45f6f58ccc5,NaN,NYCHHS: CT106820171403865,Polly Dodge Early Learning Center,538,West 55 Street,538 West 55 Street,New York,Manhattan,...,NYC Administration for Childrens Services,NYCACS,NaN,:,:,:,hhs_facilities_financialscontracts,NaN,NaN,0101000020E61000004C981A9F777F52C07DF58B995862...


#### Data snippet for example 2

In [5]:
harlem = facilities[facilities['bin']=='1089330']
harlem

,uid,hash,idold,idagency,facname,addressnum,streetname,address,city,boro,...,overagency,overabbrev,datasource,dataname,datalink,datadate,pgtable,uid_merged,hash_merged,geom
2175,36599,28dbba288a76e7fd2f49e23563a76837,NaN,NaN,"Harlem Children's Zone, Inc",245,West 129 Street,245 West 129 Street,New York,Manhattan,...,NYC Department of Youth and Community Development,NYCDYCD,NYCDYCD,NYCDYCD: COMPASS Program Locations,NYCDYCD:,NYCDYCD: 2016-12-19,dycd_facilities_compass,NaN,NaN,0101000020E6100000C3193DCCAE7C52C0237D82060268...
3845,36588,2c19411c590dc4c47b1a7fc0baba2434,NaN,NYSED: 310500860864,Harlem Children's Zone Promise,245,West 129 Street,245 West 129 Street,New York,Manhattan,...,NYC Department of Education;NYS Education Depa...,NYCDOE;NYSED,NYSED,NYSED: Listings - Active Institutions with GIS...,NYSED: https://portal.nysed.gov/discoverer/app...,NYSED: 2017-01-23,nysed_facilities_activeinstitutions,NaN,NaN,0101000020E6100000C3193DCCAE7C52C0237D82060268...
26698,60618,739decb93c19ecf56a167e35b849573b,NaN,NYCHHS: 983,Hcz Promise Academy I Charter School,245,West 129 Street,245 West 129 Street,New York,Manhattan,...,NYC Department of Youth and Community Development,NYCDYCD,NYCHHS,NYCHHS: HHS Accelerator - Proposals,NYCHHS:,NYCHHS: 2016-07-26,hhs_facilities_proposals,NaN,NaN,0101000020E6100000C3193DCCAE7C52C0237D82060268...


#### Data snippet for example 3

In [6]:
bronx = facilities[facilities['bbl']=='2027770417']
bronx

,uid,hash,idold,idagency,facname,addressnum,streetname,address,city,boro,...,overagency,overabbrev,datasource,dataname,datalink,datadate,pgtable,uid_merged,hash_merged,geom
12351,32426,15fe69f17065a70c4d985f25aff6c1db,NaN,NaN,Bronx City Recycling Inc,1390,Viele Avenue,1390 Viele Avenue,Bronx,Bronx,...,NYC Business Integrity Commission,NYCBIC,NYCBIC,NYCBIC: Approved licensees and registrants for...,NYCBIC: https://data.cityofnewyork.us/Business...,NYCBIC: 2014-09-05,bic_facilities_tradewaste,NaN,NaN,0101000020E6100000504FA20C517852C05585D3DC5367...
13578,32427,06a0a552e95dc492a78789c238f50e08,NaN,NYSDEC: ?03W88,Bronx City Recycling (1390 Viele Ave),1390,Viele Avenue,1390 Viele Avenue,Bronx,Bronx,...,NYS Department of Environmental Conservation,NYSDEC,NYSDEC,NYSDEC: Solid Waste Management Facilities,NYSDEC: https://data.ny.gov/Energy-Environment...,NYSDEC: 2017-01-09,nysdec_facilities_solidwaste,NaN,NaN,0101000020E6100000504FA20C517852C05585D3DC5367...


### B.  Eliminate rows that are an exact duplicate of another row.

We now eliminate all the rows that exhibit a 100% match in the fields containing facility name, address and idagency

In [7]:
#Filter out duplicate rows
cleaned = facilities.drop_duplicates(subset=['facname','address','idagency'])
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33571 entries, 0 to 35573
Data columns (total 47 columns):
uid            33571 non-null int64
hash           33571 non-null object
idold          182 non-null object
idagency       23634 non-null object
facname        33567 non-null object
addressnum     31275 non-null object
streetname     31275 non-null object
address        33046 non-null object
city           33545 non-null object
boro           33571 non-null object
borocode       33571 non-null int64
zipcode        33436 non-null float64
latitude       33571 non-null float64
longitude      33571 non-null float64
xcoord         33571 non-null float64
ycoord         33571 non-null float64
bin            29522 non-null object
bbl            32255 non-null object
commboard      33404 non-null float64
council        33571 non-null int64
censtract      33571 non-null int64
nta            33404 non-null object
facdomain      33571 non-null object
facgroup       33571 non-null object
fac

Thus by dropping duplicate rows, we have managed to eliminate __2003 rows__

We will now drop rows that have null (missing) values for facility name and address

### C. Eliminate rows that do not have a name or address for a facility

In [8]:
cleaned = cleaned.dropna(subset=['facname','address'])
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33043 entries, 0 to 35573
Data columns (total 47 columns):
uid            33043 non-null int64
hash           33043 non-null object
idold          182 non-null object
idagency       23171 non-null object
facname        33043 non-null object
addressnum     31273 non-null object
streetname     31273 non-null object
address        33043 non-null object
city           33027 non-null object
boro           33043 non-null object
borocode       33043 non-null int64
zipcode        32993 non-null float64
latitude       33043 non-null float64
longitude      33043 non-null float64
xcoord         33043 non-null float64
ycoord         33043 non-null float64
bin            29513 non-null object
bbl            32225 non-null object
commboard      32961 non-null float64
council        33043 non-null int64
censtract      33043 non-null int64
nta            32961 non-null object
facdomain      33043 non-null object
facgroup       33043 non-null object
fac

Dropping rows with missing values for facility name and address has helped eliminate another __528 rows__

Now let's see what the result of this initial cleaning has been on the 3 examples

### D. Evaluate cleaned data for the same 3 examples

We first add a new column 'fullName' to our 'cleaned' dataset.

Why?? That will be revealed shortly in the next step

In [9]:
cleaned['fullName'] = cleaned['facname'] + cleaned['address'] + " " + cleaned['city']

In [10]:
polly1 = cleaned[cleaned['bin']=='1086807']
polly1

,uid,hash,idold,idagency,facname,addressnum,streetname,address,city,boro,...,overabbrev,datasource,dataname,datalink,datadate,pgtable,uid_merged,hash_merged,geom,fullName
5815,37687,27ae6e2641c1a5d1bd44e8ded81ce2d9,NaN,NYCDOE: MAUT;NYCACS: 704704,YWCA - NYC Polly Dodge ELC,538,West 55 Street,538 West 55 Street,New York,Manhattan,...,NYCACS;NYCDOE,NaN,:,:,:,acs_facilities_daycareheadstart;doe_facilities...,c7cbf9d4e16bd174521ac18ebcae7ff2,c7cbf9d4e16bd174521ac18ebcae7ff2,0101000020E61000004C981A9F777F52C07DF58B995862...,YWCA - NYC Polly Dodge ELC538 West 55 Street...
6233,49518,84603ad08140caef760a522bd8635005,NaN,NYCHHS: CT106820171403865,Polly Dodge Early Learning Center,538,West 55 Street,538 West 55 Street,New York,Manhattan,...,NYCACS,NaN,:,:,:,hhs_facilities_financialscontracts,NaN,NaN,0101000020E61000004C981A9F777F52C07DF58B995862...,Polly Dodge Early Learning Center538 West 55...
12156,37691,4d5620b62186d283b6e3b938400f6997,NaN,NYCDOHMH: DC3066,Young Women's Christian Association Of The Cit...,538,West 55 Street,538 West 55 Street,New York,Manhattan,...,NYCDOHMH,NYCDOHMH,NYCDOHMH: DOHMH Childcare Center Inspections,NYCDOHMH: https://data.cityofnewyork.us/Health...,NYCDOHMH: 2017-01-23,dohmh_facilities_daycare,NaN,NaN,0101000020E61000004C981A9F777F52C07DF58B995862...,Young Women's Christian Association Of The Cit...


In [11]:
harlem1 = cleaned[cleaned['bin']=='1089330']
harlem1

,uid,hash,idold,idagency,facname,addressnum,streetname,address,city,boro,...,overabbrev,datasource,dataname,datalink,datadate,pgtable,uid_merged,hash_merged,geom,fullName
2175,36599,28dbba288a76e7fd2f49e23563a76837,NaN,NaN,"Harlem Children's Zone, Inc",245,West 129 Street,245 West 129 Street,New York,Manhattan,...,NYCDYCD,NYCDYCD,NYCDYCD: COMPASS Program Locations,NYCDYCD:,NYCDYCD: 2016-12-19,dycd_facilities_compass,NaN,NaN,0101000020E6100000C3193DCCAE7C52C0237D82060268...,"Harlem Children's Zone, Inc245 West 129 Stree..."
3845,36588,2c19411c590dc4c47b1a7fc0baba2434,NaN,NYSED: 310500860864,Harlem Children's Zone Promise,245,West 129 Street,245 West 129 Street,New York,Manhattan,...,NYCDOE;NYSED,NYSED,NYSED: Listings - Active Institutions with GIS...,NYSED: https://portal.nysed.gov/discoverer/app...,NYSED: 2017-01-23,nysed_facilities_activeinstitutions,NaN,NaN,0101000020E6100000C3193DCCAE7C52C0237D82060268...,Harlem Children's Zone Promise245 West 129 St...
26698,60618,739decb93c19ecf56a167e35b849573b,NaN,NYCHHS: 983,Hcz Promise Academy I Charter School,245,West 129 Street,245 West 129 Street,New York,Manhattan,...,NYCDYCD,NYCHHS,NYCHHS: HHS Accelerator - Proposals,NYCHHS:,NYCHHS: 2016-07-26,hhs_facilities_proposals,NaN,NaN,0101000020E6100000C3193DCCAE7C52C0237D82060268...,Hcz Promise Academy I Charter School245 West ...


In [12]:
bronx1 = cleaned[cleaned['bbl']=='2027770417']
bronx1

,uid,hash,idold,idagency,facname,addressnum,streetname,address,city,boro,...,overabbrev,datasource,dataname,datalink,datadate,pgtable,uid_merged,hash_merged,geom,fullName
12351,32426,15fe69f17065a70c4d985f25aff6c1db,NaN,NaN,Bronx City Recycling Inc,1390,Viele Avenue,1390 Viele Avenue,Bronx,Bronx,...,NYCBIC,NYCBIC,NYCBIC: Approved licensees and registrants for...,NYCBIC: https://data.cityofnewyork.us/Business...,NYCBIC: 2014-09-05,bic_facilities_tradewaste,NaN,NaN,0101000020E6100000504FA20C517852C05585D3DC5367...,Bronx City Recycling Inc1390 Viele Avenue Bronx
13578,32427,06a0a552e95dc492a78789c238f50e08,NaN,NYSDEC: ?03W88,Bronx City Recycling (1390 Viele Ave),1390,Viele Avenue,1390 Viele Avenue,Bronx,Bronx,...,NYSDEC,NYSDEC,NYSDEC: Solid Waste Management Facilities,NYSDEC: https://data.ny.gov/Energy-Environment...,NYSDEC: 2017-01-09,nysdec_facilities_solidwaste,NaN,NaN,0101000020E6100000504FA20C517852C05585D3DC5367...,Bronx City Recycling (1390 Viele Ave)1390 Viel...


### While cleaning out duplicate entries hasn't helped with example 2 and 3, we notice its adavantage in example 1, where 2 identical rows have been eliminated.

### E. Test fuzzywuzzy library implementation on the cleaned data for the same 3 examples

[FuzzyWuzzy](http://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/) is a Python library created by SeatGeek and open sourced in 2011. It is primarily used to check for similarity between two strings.

It is here that we put the newly created column 'fullName' to use. A string containing the name, addres and city of each facility (row in this case) is created and stored in that column.

Let's understand how FuzzyWuzzy works by implementing it on cleaned snippets of our 3 examples 

In [13]:
from fuzzywuzzy import fuzz

C:\Users\ManushiM\Anaconda\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### Defining a function that returns similarity scores in the form of matrix, based on values ofcolumn 'fullName' for any dataframe similar to 'cleaned'

In [14]:
"""Generates similarity matrix for given dataframe"""
def generateSimilarityMatrix(df):

    # Create an empty dataframe to store the results
    df_similarity = pd.DataFrame(index=df['uid'].values, columns=df['uid'].values, dtype='float')
    

    # Calculate the similarity score for every record pair
    # And insert it into the empty dataframe
    for i in df_similarity.index:        
        rowname = str(df[df['uid'] == i]['fullName'].values[0])
        for c in df_similarity.columns:
            colname = str(df[df['uid'] == c]['fullName'].values[0])
            similarityScore = fuzz.partial_ratio(rowname, colname)/100.0
            df_similarity.ix[i][c] = similarityScore
    return df_similarity

#### For example 1

In [15]:
#Similarity matrix for polly1
sim1 = generateSimilarityMatrix(polly1)
sim1

,37687,49518,37691
37687,1.00,0.67,0.71
49518,0.67,1.00,0.55
37691,0.71,0.55,1.00


In [16]:
#Finding corresponding rows in polly1
polly1[['uid', 'facname', 'address', 'city']]

,uid,facname,address,city
5815,37687,YWCA - NYC Polly Dodge ELC,538 West 55 Street,New York
6233,49518,Polly Dodge Early Learning Center,538 West 55 Street,New York
12156,37691,Young Women's Christian Association Of The Cit...,538 West 55 Street,New York


We notice the highest similarity score of 0.71 for __'YWCA - NYC Polly Dodge ELC'__ and __'Young Women's Christian Association Of The City of New York'__, which is understandable since one name is the acronym for the other.

#### For example 2

In [17]:
#Similarity matrix for harlem1
sim2 = generateSimilarityMatrix(harlem1)
sim2

,36599,36588,60618
36599,1.00,0.88,0.67
36588,0.88,1.00,0.58
60618,0.67,0.58,1.00


In [18]:
#Finding corresponding rows in harlem1
harlem1[['uid', 'facname', 'address', 'city']]

,uid,facname,address,city
2175,36599,"Harlem Children's Zone, Inc",245 West 129 Street,New York
3845,36588,Harlem Children's Zone Promise,245 West 129 Street,New York
26698,60618,Hcz Promise Academy I Charter School,245 West 129 Street,New York


We notice the highest similarity score of 0.71 for __'Harlem Children's Zone, Inc'__ and __'Harlem Children's Zone Promise'__, based on higher similarity between names.

#### For example 3

In [19]:
#Similarity matrix for bronx1
sim3 = generateSimilarityMatrix(bronx1)
sim3

,32426,32427
32426,1.00,0.77
32427,0.77,1.00


In [20]:
#Finding corresponding rows in bronx1
bronx1[['uid', 'facname', 'address', 'city']]

,uid,facname,address,city
12351,32426,Bronx City Recycling Inc,1390 Viele Avenue,Bronx
13578,32427,Bronx City Recycling (1390 Viele Ave),1390 Viele Avenue,Bronx


In this example we have just 2 values to compare. A high similarity score of 0.77 __ensures that multiple facilties with slight discrepancies in the facility names can also be detected__.

### Conclusion:

* FuzzyWuzzy can be considered a reliable metric to gauge similarity of records within a cleaned dataset.

* Another interesting implementation of the FuzzyWuzzy library by the same team is in [Will Geary's notebook](https://github.com/willgeary/detectingduplicates/blob/master/Similarity%20Heatmaps%20Notebook.ipynb) that tests similarity for a larger subset without eliminating duplicates from the dataset.

### Team:

1. [Will Geary](http://willgeary.github.io/)
2. [Manushi Majumdar](https://github.com/ManushiM/)
3. [Ian Stuart](https://www.linkedin.com/in/ianwstuart/)
4. [Pooneh Famili](https://www.linkedin.com/in/pooneh-famili-ba007626/)